# О задании

## Тема
Объединение таблиц и сложные выборки

## Цель домашнего задания
- научиться работать с интервальными таблицы, уметь их джойнить с другими таблицами;
- научиться обрабатывать транзакции относительно какого-то промежутка времени.
 

## Условия
Дано 2 CSV-файла: один с транзакциями за период 01.01.2023 – 20.04.2023, второй с информацией о клиентах.

**За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’**

Необходимо в Jupyter- ноутбуке выполнить следующие пункты, используя SQLite:

<font size=4>[**Шаг 1**](#Шаг-1)</font>

(Балл - 2) Необходимо скачать CSV-файл - «[transactions_for_dz2.csv](https://drive.google.com/file/d/1GsKxPQ3cp1_uuyoLoAQLlsHTLEbXOMIo/view?usp=share_link)», создать таблицу *transaction_bd* со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции

(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции

(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента

(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала

(4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции

Также необходимо скачать второй CSV-файл - «[client_info.csv](https://drive.google.com/file/d/1oFzZfwHBtP0NOulBYlp4osIY5NIPIdn-/view?usp=share_link)», создать таблицу *customer_bd* со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:

(0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента

(1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте

(2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте

(3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента

(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождение клиента

**Примечание для следующих шагов:** сумма транзакций не может быть null-ом, это всегда число.

<font size=4>[**Шаг 2**](#Шаг-2)</font>

**(Балл - 1 за каждый пункт)** Написать скрипты:

- a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
- b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
- c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.
- d. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.
- e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

<font size=4>[**Шаг 3**](#Шаг-3)</font>

**(Балл - 3)** Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
 

## Сроки выполнения задания
До 09.04.2023, 23:59.

In [1]:
import pandas as pd
import sqlite3

## Шаг 1

In [32]:
transaction_bd = pd.read_csv('transactions_for_dz2.csv', parse_dates=['TX_DATETIME'])

In [33]:
transaction_bd.head(3)

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94


In [34]:
transaction_bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   TRANSACTION_ID  1048575 non-null  int64         
 1   TX_DATETIME     1048575 non-null  datetime64[ns]
 2   CUSTOMER_ID     1048575 non-null  int64         
 3   TERMINAL_ID     1048575 non-null  int64         
 4   TX_AMOUNT       1048575 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 40.0 MB


Создадим функции для подключения к базе данных, подключимся к ней, создадим таблицу и наполним ее данными

In [35]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [36]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [37]:
connection = create_connection('database.db')

Connection to SQLite DB successful


In [38]:
sql_create_table = '''
                        CREATE TABLE IF NOT EXISTS transaction_bd (
                                            "TRANSACTION_ID" INTEGER,
                                            "TX_DATETIME" NUMERIC,
                                            "CUSTOMER_ID" INTEGER,
                                            "TERMINAL_ID" INTEGER,
                                            "TX_AMOUNT" REAL
                                                        )
                     '''

In [39]:
execute_query(connection, sql_create_table)

Query executed successfully


In [40]:
transaction_bd.to_sql('transaction_bd', connection, if_exists='replace', index=False)

1048575

In [41]:
connection.close()

## Шаг 2

## Шаг 3